# First load os and sys so I can update the sys.path with new functions

change the luminosity to 80 /fb

generate the 3 plots as in aewol paper.

In [1]:
import os
import sys

In [2]:
#take the paths to the functions we nedd
module_path = os.path.abspath(os.path.join('./pandasPlotting/'))
module2_path = os.path.abspath(os.path.join('./MlClasses/'))
module3_path = os.path.abspath(os.path.join('./MlFunctions/'))

In [3]:
# this part will include in the sys.path variables the paths for our new functions
if [module_path, module2_path, module3_path] not in sys.path:
    sys.path.append(module_path)

In [4]:
# here we are going to load what we will need, keras + tensorflow, plot functions, etc..
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import math
import time

from keras import callbacks

from pandasPlotting.Plotter import Plotter
from pandasPlotting.dfFunctions import expandArrays
from pandasPlotting.dtFunctions import featureImportance

from MlClasses.MlData import MlData
from MlClasses.Bdt import Bdt
from MlClasses.Dnn import Dnn
from MlClasses.ComparePerformances import ComparePerformances

from MlFunctions.DnnFunctions import significanceLoss,significanceLossInvert,significanceLoss2Invert ,significanceLossInvertSqrt,significanceFull,asimovSignificanceLoss,asimovSignificanceLossInvert,asimovSignificanceFull,truePositive,falsePositive

from linearAlgebraFunctions import gram,addGramToFlatDF

Using TensorFlow backend.


In [5]:
# I don't have patience to training 200 epochs ¯\_(ツ)_/¯
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5)

In [6]:
#load our data files
signal=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0L-inclusive/zh0p030lfeb.csv",sep='\s+',engine='python')
bkgd=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0L-inclusive/zhsm0lfeb.csv",sep='\s+',engine='python')


In [7]:
#combine them into one dataset
combined = pd.concat([signal,bkgd]).sample(frac=1)

In [8]:
print(combined.keys())

Index(['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh',
       'dphib1met', 'signal'],
      dtype='object')


In [9]:
# change thes vars depend on which dataset you are loading, I will implement a better solution.
chosenVars = {
            # #A vanilla analysis with HL variables and lead 3 jets
            '0L':['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh', 'dphib1met', 'signal'],
            #'2L':['ptb1', 'ptb2', 'ptl1', 'ptl2', 'pth', 'ptz', 'etah', 'phih',
            #     'deltarll', 'deltarbl', 'mtvh', 'ptvh', 'dphil1b1', 'dphil1b2', 'signal']
}

In [10]:
trainedModels={}

In [11]:
#needed to plot asimov significane
asimovSigLossSysts=[0.01,0.05,0.1,0.2,0.3,0.4,0.5]

In [12]:
# here I have included one archtecture I got from my ES scan, pls comment my entry and use dnn_batch4096 instead.
dnnConfigs={
    'dnn_ZH_0L_cHW_0d03':{'epochs':20,'batch_size':8192,'dropOut':None,
                                      'l2Regularization':None,'hiddenLayers':[1.0],
                                      'optimizer':'adam', 'activation':'relu'}     
        }

In [13]:
#bsmlike signal cHW 0.03: 11.867fb, bg:0.89
#inclusive signal cHW 0.001: 2.198fb, bg:2.03fb
#inclusive signal cHW 0.01: 4.553fb, bg:2.03fb
#inclusive signal cHW 0.03: 14fb, bg:2.03fb

lumi=80. #luminosity in /fb
expectedSignal=14*lumi 
expectedBkgd=2.03*lumi #cross section of ttbar sample in fb times efficiency measured by Marco
systematic=0.5 #systematic for the asimov signficance

In [14]:
# I'm only running the DNN with binary cross entropy, letter I will include the other options.
# so far it running perfect, I'm testing with their custom loss function.
for varSetName,varSet in chosenVars.items():
    #Pick out the expanded arrays
    columnsInDataFrame = []
    for k in combined.keys():
        for v in varSet:
            #Little trick to ensure only the start of the string is checked
            if varSetName is '0L':
                if ' '+v+' ' in ' '+k+' ': columnsInDataFrame.append(k)
            elif ' '+v in ' '+k: columnsInDataFrame.append(k)


    #Select just the features we're interested in
    #For now setting NaNs to 0 for compatibility
    combinedToRun = combined[columnsInDataFrame].copy()
    combinedToRun.fillna(0,inplace=True)
    
    combinedToRun.index = np.arange(0,200000)
    mlData = MlData(combinedToRun,'signal')

    mlData.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

    for name,config in dnnConfigs.items():
        dnn = Dnn(mlData,'testPlots/mlPlots/variableBatchSigLossInvert/0L'+varSetName+'/'+name)

        dnn.setup(hiddenLayers=config['hiddenLayers'],dropOut=config['dropOut'],
                  l2Regularization=config['l2Regularization'],
                loss=significanceLossInvert(expectedSignal,expectedBkgd),
                extraMetrics=[
                    significanceLoss(expectedSignal,expectedBkgd),significanceFull(expectedSignal,expectedBkgd),
                    asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),truePositive,falsePositive
                ])


        for batch_size in [128,512,1024,2048,4096,8192,16384]:
            dnn.fit(epochs=20,batch_size=batch_size,callbacks=[earlyStopping])
            dnn.diagnostics(batchSize=4096,subDir='batchSize'+str(batch_size))

        dnn.diagnostics(batchSize=4096)
        dnn.makeHepPlots(expectedSignal,expectedBkgd,systematic,makeHistograms=False)

        trainedModels[varSetName+'_variableBatchSigLossInvert_'+name]=dnn


/home/felipe/hepML/MlClasses/Dnn.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.history = self.model.fit(self.data.X_train.as_matrix(), self.data.y_train.as_matrix(), sample_weight=self.data.weights_train,
/home/felipe/hepML/MlClasses/Dnn.py:102: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  validation_data=(self.data.X_test.as_matrix(),self.data.y_test.as_matrix(),self.data.weights_test),


Train on 140000 samples, validate on 60000 samples
Epoch 1/20
140000/140000 [==============================] - 3s 19us/step - loss: 0.0012 - acc: 0.5095 - sigLoss: -848.0232 - significance: 30.2799 - asimovSignificance: 6.2947 - truePositive: 0.9415 - falsePositive: 0.9211 - val_loss: 0.0010 - val_acc: 0.5008 - val_sigLoss: -960.5424 - val_significance: 31.2756 - val_asimovSignificance: 6.1691 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 2/20
140000/140000 [==============================] - 2s 14us/step - loss: 0.0010 - acc: 0.4997 - sigLoss: -969.9007 - significance: 31.2756 - asimovSignificance: 6.1691 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: 0.0010 - val_acc: 0.5008 - val_sigLoss: -974.3738 - val_significance: 31.2756 - val_asimovSignificance: 6.1691 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 3/20
140000/140000 [==============================] - 2s 14us/step - loss: 0.0010 - acc: 0.4997 - sigLoss: -975.8910 - significance: 31.2756 

/home/felipe/hepML/MlClasses/Dnn.py:221: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_test.as_matrix(), y_test.as_matrix(), sample_weight=weights_test, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:227: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_test.as_matrix()),self.model.predict(X_test.as_matrix()),y_test,f)


140000/140000 [==============================] - 0s 0us/step


/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/felipe/hepML/MlClasses/Dnn.py:232: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_train.as_matrix(), y_train.as_matrix(), sample_weight=weights_train, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_train.as_matrix()),self.model.predict(X_train.as_matrix()),y_train,f)
/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sample

...

60000/60000 [==============================] - 0s 0us/step


/home/felipe/hepML/MlClasses/Dnn.py:221: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_test.as_matrix(), y_test.as_matrix(), sample_weight=weights_test, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:227: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_test.as_matrix()),self.model.predict(X_test.as_matrix()),y_test,f)


140000/140000 [==============================] - 0s 0us/step


/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/felipe/hepML/MlClasses/Dnn.py:232: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_train.as_matrix(), y_train.as_matrix(), sample_weight=weights_train, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_train.as_matrix()),self.model.predict(X_train.as_matrix()),y_train,f)
/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sample

TypeError: 'float' object is not iterable